<a href="https://colab.research.google.com/github/datasigntist/tensorflowCodes/blob/master/tensorflow101_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import functools 
import numpy as np 
import tensorflow as tf 
import pandas as pd

LABEL_COLUMN = "Survived"

TRAIN_DATA_URL=  "https://raw.githubusercontent.com/datasigntist/datasetsForTraining/master/train.csv" 

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL) 

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

titanicDataSet = get_dataset(train_file_path)

class extractFeaturesFromName(object):
  def __call__(self, features, labels):
    namesOfPassengers = features['Name']
    lastName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(feat,",")[0]) ,namesOfPassengers)
    firstName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[1]) ,namesOfPassengers)
    title_feature  = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[0]) ,namesOfPassengers)
    features['last_name'] = lastName_feature
    features['first_name'] = firstName_feature
    features['title_feature'] = title_feature
    return features, labels

class replaceMissingValuesInAge(object):
  def __init__(self, meanAge):
    self.meanAge = meanAge

  def __call__(self, features, labels):
    ageOfPassengers = features['Age']
    ageOfPassengers = tf.map_fn(lambda age: (age if age>0 else self.meanAge) ,ageOfPassengers)
    features['Age'] = ageOfPassengers
    return features, labels

desc = pd.read_csv(train_file_path)["Age"].describe()
MEAN = np.float32(np.array(desc.T['mean']))

newtitanicDataSet = titanicDataSet.map(extractFeaturesFromName())
newtitanicDataSet = titanicDataSet.map(replaceMissingValuesInAge(MEAN))

show_batch(newtitanicDataSet)

PassengerId         : [ 18 776 241 744 134]
Pclass              : [2 3 3 3 2]
Name                : [b'Williams, Mr. Charles Eugene'
 b'Myhrman, Mr. Pehr Fabian Oliver Malkolm' b'Zabour, Miss. Thamine'
 b'McNamee, Mr. Neal' b'Weisz, Mrs. Leopold (Mathilde Francoise Pede)']
Sex                 : [b'male' b'male' b'female' b'male' b'female']
Age                 : [29.699118 18.       29.699118 24.       29.      ]
SibSp               : [0 0 1 1 1]
Parch               : [0 0 0 0 0]
Ticket              : [b'244373' b'347078' b'2665' b'376566' b'228414']
Fare                : [13.      7.75   14.4542 16.1    26.    ]
Cabin               : [b'' b'' b'' b'' b'']
Embarked            : [b'S' b'S' b'C' b'S' b'S']
